# House Sales Predictions Report

### Dependencies

In [39]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt

#Training and testing dependencies
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.model_selection import GridSearchCV


### Upload datasource

Datasources are composed of a train dataset and a test one, but I wil concatenate this two files, in order to transform and select columns the same way for the two files. Before training, I will split the file into a the training one and testing one which will be submitted.

In [40]:
##Import the file, reindex them in order to concatenate them 
df_train = pd.read_csv("train.csv")
df_train.index = pd.RangeIndex(1,1+len(df_train.index))
saleprice=df_train
df_train.drop(["SalePrice"], axis=1)
df_test=pd.read_csv("test.csv")
df=pd.concat([df_train,df_test])
df.index = pd.RangeIndex(1,1+len(df.index))
display(df.head())

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,BsmtFinType2,BsmtFullBath,BsmtHalfBath,BsmtQual,BsmtUnfSF,CentralAir,Condition1,Condition2,Electrical,EnclosedPorch,ExterCond,ExterQual,Exterior1st,Exterior2nd,Fence,FireplaceQu,Fireplaces,Foundation,FullBath,Functional,GarageArea,GarageCars,GarageCond,GarageFinish,GarageQual,GarageType,GarageYrBlt,GrLivArea,HalfBath,Heating,HeatingQC,HouseStyle,Id,KitchenAbvGr,KitchenQual,LandContour,LandSlope,LotArea,LotConfig,LotFrontage,LotShape,LowQualFinSF,MSSubClass,MSZoning,MasVnrArea,MasVnrType,MiscFeature,MiscVal,MoSold,Neighborhood,OpenPorchSF,OverallCond,OverallQual,PavedDrive,PoolArea,PoolQC,RoofMatl,RoofStyle,SaleCondition,SalePrice,SaleType,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold
1,856,854,0,NaN,3,1Fam,TA,No,706.0,0.0,GLQ,Unf,1.0,0.0,Gd,150.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,NaN,0,PConc,2,Typ,548.0,2.0,TA,RFn,TA,Attchd,2003.0,1710,1,GasA,Ex,2Story,1,1,Gd,Lvl,Gtl,8450,Inside,65.0,Reg,0,60,RL,196.0,BrkFace,NaN,0,2,CollgCr,61,5,7,Y,0,NaN,CompShg,Gable,Normal,208500.0,WD,0,Pave,8,856.0,AllPub,0,2003,2003,2008
2,1262,0,0,NaN,3,1Fam,TA,Gd,978.0,0.0,ALQ,Unf,0.0,1.0,Gd,284.0,Y,Feedr,Norm,SBrkr,0,TA,TA,MetalSd,MetalSd,NaN,TA,1,CBlock,2,Typ,460.0,2.0,TA,RFn,TA,Attchd,1976.0,1262,0,GasA,Ex,1Story,2,1,TA,Lvl,Gtl,9600,FR2,80.0,Reg,0,20,RL,0.0,None,NaN,0,5,Veenker,0,8,6,Y,0,NaN,CompShg,Gable,Normal,181500.0,WD,0,Pave,6,1262.0,AllPub,298,1976,1976,2007
3,920,866,0,NaN,3,1Fam,TA,Mn,486.0,0.0,GLQ,Unf,1.0,0.0,Gd,434.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,TA,1,PConc,2,Typ,608.0,2.0,TA,RFn,TA,Attchd,2001.0,1786,1,GasA,Ex,2Story,3,1,Gd,Lvl,Gtl,11250,Inside,68.0,IR1,0,60,RL,162.0,BrkFace,NaN,0,9,CollgCr,42,5,7,Y,0,NaN,CompShg,Gable,Normal,223500.0,WD,0,Pave,6,920.0,AllPub,0,2001,2002,2008
4,961,756,0,NaN,3,1Fam,Gd,No,216.0,0.0,ALQ,Unf,1.0,0.0,TA,540.0,Y,Norm,Norm,SBrkr,272,TA,TA,Wd Sdng,Wd Shng,NaN,Gd,1,BrkTil,1,Typ,642.0,3.0,TA,Unf,TA,Detchd,1998.0,1717,0,GasA,Gd,2Story,4,1,Gd,Lvl,Gtl,9550,Corner,60.0,IR1,0,70,RL,0.0,None,NaN,0,2,Crawfor,35,5,7,Y,0,NaN,CompShg,Gable,Abnorml,140000.0,WD,0,Pave,7,756.0,AllPub,0,1915,1970,2006
5,1145,1053,0,NaN,4,1Fam,TA,Av,655.0,0.0,GLQ,Unf,1.0,0.0,Gd,490.0,Y,Norm,Norm,SBrkr,0,TA,Gd,VinylSd,VinylSd,NaN,TA,1,PConc,2,Typ,836.0,3.0,TA,RFn,TA,Attchd,2000.0,2198,1,GasA,Ex,2Story,5,1,Gd,Lvl,Gtl,14260,FR2,84.0,IR1,0,60,RL,350.0,BrkFace,NaN,0,12,NoRidge,84,5,8,Y,0,NaN,CompShg,Gable,Normal,250000.0,WD,0,Pave,9,1145.0,AllPub,192,2000,2000,2008


## Features engineering
1. What are the number of missing values for each column?
   * Selecting and Deleting columns containing more than 5% of missing values
2. Text column: deleting every column with missing values
3. Numerical columns: For columns with missing values, fill in with the most common value in that column

In [41]:
print("##########################################################################################################")
print("Dataset size {}".format(df.shape))
print('')

## Series object: column name -> number of missing values
print("##########################################################################################################")
print("1: Number of missing values in each column?")
print('')
num_missing = df.isnull().sum()
print(num_missing)

# Filter Series to columns containing >5% missing values
drop_missing_cols = num_missing[(num_missing > len(df)/20)].sort_values()
print('#########################################################################################################')
print("Select columns containing more than 5% of missing values")
print('')
print(drop_missing_cols)

# Drop those columns from the data frame. Note the use of the .index accessor
print('')

df = df.drop(drop_missing_cols.index, axis=1)
print("Delete columns selected above, our new dataset size is {}".format(df.shape))

print('')
print("#########################################################################################################")
print("2: Text columns: Drop any with 1 or more missing values for now, dataset size : {}".format(df.shape))
## Series object: column name -> number of missing values
text_mv_counts = df.select_dtypes(include=['object']).isnull().sum().sort_values(ascending=False)
## Filter Series to columns containing *any* missing values
drop_missing_cols_2 = text_mv_counts[text_mv_counts > 0]
df = df.drop(drop_missing_cols_2.index, axis=1)
print(drop_missing_cols_2)
print('')
print("#########################################################################################################")
print("3: Numerical columns: For columns with missing values, fill in with the most common value in that column" )
## Compute column-wise missing value counts
num_missing = df.select_dtypes(include=['int', 'float']).isnull().sum()
print(num_missing)
fixable_numeric_cols = num_missing[(num_missing < len(df)/20) & (num_missing > 0)].sort_values()
fixable_numeric_cols
## Compute the most common value for each column in `fixable_nmeric_missing_cols`.
replacement_values_dict = df[fixable_numeric_cols.index].mode().to_dict(orient='records')[0]
print("replacement values")
print(replacement_values_dict)
## Use `pd.DataFrame.fillna()` to replace missing values.
df = df.fillna(replacement_values_dict)


print("dataset size {}".format(df.shape))

print("Verify that every column has 0 missing values")
print('')
print("Number of missing values")
print(df.isnull().sum())

##########################################################################################################
Dataset size (2919, 81)

##########################################################################################################
1: Number of missing values in each column?

1stFlrSF            0
2ndFlrSF            0
3SsnPorch           0
Alley            2721
BedroomAbvGr        0
BldgType            0
BsmtCond           82
BsmtExposure       82
BsmtFinSF1          1
BsmtFinSF2          1
BsmtFinType1       79
BsmtFinType2       80
BsmtFullBath        2
BsmtHalfBath        2
BsmtQual           81
BsmtUnfSF           1
CentralAir          0
Condition1          0
Condition2          0
Electrical          1
EnclosedPorch       0
ExterCond           0
ExterQual           0
Exterior1st         1
Exterior2nd         1
Fence            2348
FireplaceQu      1420
Fireplaces          0
Foundation          0
FullBath            0
                 ... 
LotShape            0
LowQualFinSF

#### Engineering Date features 

I create new features to better capture the information of the dates. Columns Years before Sale and Years Since Remod would be added.

In [42]:
print("###########################################################################################################")
print("Creation of a new column that summarize the year sold information")
print('')
years_sold = df['YrSold'] - df['YearBuilt']
years_sold[years_sold < 0]


years_since_remod = df['YrSold'] - df['YearRemodAdd']
print(years_since_remod[years_since_remod < 0])

print("Adding the columns 'Years Before Sale' et 'Years Since Remod' ")
print('')
## Create new columns
df['Years Before Sale'] = years_sold
df['Years Since Remod'] = years_since_remod

## Drop rows with negative values for both of these new features
print("Years Since Remod and Years Before Sale negativ values are taking the 0 value")
print(years_since_remod[years_since_remod < 0].index)

df.loc[df['Years Since Remod']<0,'Years Since Remod'] = 0
df.loc[df['Years Before Sale']<0,'Years Before Sale'] = 0
print("Deleting the columns 'YearBuilt' et 'YearRemodAdd' ")
## No longer need original year columns
df = df.drop(["YearBuilt", "YearRemodAdd"], axis = 1)
print("The final dataset size is {}".format(df.shape))

###########################################################################################################
Creation of a new column that summarize the year sold information

524    -1
2296   -1
2550   -2
dtype: int64
Adding the columns 'Years Before Sale' et 'Years Since Remod' 

Years Since Remod and Years Before Sale negativ values are taking the 0 value
Int64Index([524, 2296, 2550], dtype='int64')
Deleting the columns 'YearBuilt' et 'YearRemodAdd' 
The final dataset size is (2919, 55)


#### Which columns need to be dropped for other reasons?

 * Which columns aren't useful for machine learning?
 * Which columns leak data about the final sale?


In [43]:
print("#################################################################################################################")
print('')
print("Drop columns that: aren't useful for ML leak data about the final sale")
## Drop columns that leak info about the final sale
df = df.drop(["MoSold", "SaleCondition", "YrSold"], axis=1)
if "SaleType" in df.columns : df=df.drop(["SaleType"] )
print("The final dataset size is {}".format(df.shape))
df_test = df.iloc[1460:]
df_train = df.iloc[:1460]
df_train["SalePrice"] = saleprice.iloc[0:1460]["SalePrice"]



#################################################################################################################

Drop columns that: aren't useful for ML leak data about the final sale
The final dataset size is (2919, 52)


C:\Users\Amine\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


## Features Selection

1. Selection of numeric column with correlation coefficient larger than 0.4 with the targeted features : "SalePrice".    Deleting the others.
2. Which categorical columns should we keep?
    * Which columns are currently numerical but need to be encoded as categorical instead 
    * If a categorical column has hundreds of unique values (or categories), I decided to keep only the one with less than 10 unique values.
    * When we dummy code this column, hundreds of columns will need to be added back to the data frame.

In [44]:
print("#################################################################################################################")
transform_df = df_train
print("After transformation (missing values treatment etc...) dataset size {}".format(transform_df.shape))
print('')
numerical_df = transform_df.select_dtypes(include=['float64','int64'])
print("Column of numeric Type dataset size {}".format(numerical_df.shape))
print('')
#Table of correlation with the taret column
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
print("Table of correlation with the targeted column 'SalePrice'")
print('')
print(abs_corr_coeffs)
print('')

## Let's only keep columns with a correlation coefficient of larger than 0.4 (arbitrary, worth experimenting later!)

print("Here is the list of numeric column with correlation column larger than 0.4")
display(abs_corr_coeffs[abs_corr_coeffs > 0.4])
print('')
print(len(df_test.columns),len(transform_df.columns))
## Drop columns with less than 0.4 correlation with SalePrice
transform_df = transform_df.drop(abs_corr_coeffs[abs_corr_coeffs < 0.4].index, axis=1)
print("After deleting low correlated column dataset size {}".format(transform_df.shape))
print(len(df_test.columns),len(transform_df.columns))
display(transform_df.head())
features_train = transform_df.drop(["SalePrice"],axis=1).columns
df_test=df_test[features_train]


## Create a list of column names from documentation that are *meant* to be categorical
nominal_features = ["PID", "MSSubClass", "MS Zoning", "Street", "Alley", "Land Contour", "LotConfig", "Neighborhood", 
                    "Condition1", "Condition2", "Bldg Type", "House Style", "RoofStyle", "RoofMatl", "Exterior1st", 
                    "Exterior2nd", "MasVnrType", "Foundation", "Heating", "CentralAir", "GarageType", 
                    "MiscFeature", "SaleType", "SaleCondition"]
print(len(df_test.columns),len(transform_df.columns))
## Which categorical columns have we still carried with us? We'll test tehse 
transform_cat_cols = []
for col in nominal_features:
    if col in transform_df.columns:
        transform_cat_cols.append(col)
print(len(df_test.columns),len(transform_df.columns))
## How many unique values in each categorical column?
uniqueness_counts = transform_df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
print(uniqueness_counts)


## Aribtrary cutoff of 10 unique values (worth experimenting)
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
print("Columns that must be deleted because of too much non unique values")
print(drop_nonuniq_cols)
transform_df = transform_df.drop(drop_nonuniq_cols, axis=1)

features_train = transform_df.drop(["SalePrice"],axis=1).columns
df_test=df_test[features_train]

## Select just the remaining text columns and convert to categorical
text_cols = transform_df.select_dtypes(include=['object'])
text_cols_test = df_test.select_dtypes(include=['object'])

for col in text_cols:
    transform_df[col] = transform_df[col].astype('category')
    df_test[col] = df_test[col].astype('category')
    
## Create dummy columns and add back to the dataframe!
transform_df = pd.concat([transform_df,pd.get_dummies(transform_df.select_dtypes(include=['category']))], axis=1)
    
df_test = pd.concat([df_test,pd.get_dummies(df_test.select_dtypes(include=['category']))], axis=1)


column_difference=list(set(transform_df.drop(["SalePrice"],axis=1).columns) - set(df_test.columns))
print('')

transform_df =transform_df.drop(column_difference,axis=1)
column_difference=list(set(transform_df.drop(["SalePrice"],axis=1).columns) - set(df_test.columns))
print("Verifying that the column of the training and testing set are the same : column difference {}".format(column_difference))


#################################################################################################################
After transformation (missing values treatment etc...) dataset size (1460, 53)

Column of numeric Type dataset size (1460, 34)

Table of correlation with the targeted column 'SalePrice'

BsmtFinSF2           0.011378
BsmtHalfBath         0.016844
MiscVal              0.021190
Id                   0.021917
LowQualFinSF         0.025606
3SsnPorch            0.044584
OverallCond          0.077856
MSSubClass           0.084284
PoolArea             0.092404
ScreenPorch          0.111447
EnclosedPorch        0.128578
KitchenAbvGr         0.135907
BedroomAbvGr         0.168213
BsmtUnfSF            0.214479
BsmtFullBath         0.227122
LotArea              0.263843
HalfBath             0.284108
OpenPorchSF          0.315856
2ndFlrSF             0.319334
WoodDeckSF           0.324413
BsmtFinSF1           0.386420
Fireplaces           0.466929
MasVnrArea           0.472614
Years Sin

Fireplaces           0.466929
MasVnrArea           0.472614
Years Since Remod    0.509096
Years Before Sale    0.523350
TotRmsAbvGrd         0.533723
FullBath             0.560664
1stFlrSF             0.605852
TotalBsmtSF          0.613581
GarageArea           0.623431
GarageCars           0.640409
GrLivArea            0.708624
OverallQual          0.790982
SalePrice            1.000000
Name: SalePrice, dtype: float64


52 53
After deleting low correlated column dataset size (1460, 32)
52 32


,1stFlrSF,BldgType,CentralAir,Condition1,Condition2,ExterCond,ExterQual,Fireplaces,Foundation,FullBath,GarageArea,GarageCars,GrLivArea,Heating,HeatingQC,HouseStyle,LandContour,LandSlope,LotConfig,LotShape,MasVnrArea,Neighborhood,OverallQual,PavedDrive,RoofMatl,RoofStyle,Street,TotRmsAbvGrd,TotalBsmtSF,Years Before Sale,Years Since Remod,SalePrice
1,856,1Fam,Y,Norm,Norm,TA,Gd,0,PConc,2,548.0,2.0,1710,GasA,Ex,2Story,Lvl,Gtl,Inside,Reg,196.0,CollgCr,7,Y,CompShg,Gable,Pave,8,856.0,5,5,208500
2,1262,1Fam,Y,Feedr,Norm,TA,TA,1,CBlock,2,460.0,2.0,1262,GasA,Ex,1Story,Lvl,Gtl,FR2,Reg,0.0,Veenker,6,Y,CompShg,Gable,Pave,6,1262.0,31,31,181500
3,920,1Fam,Y,Norm,Norm,TA,Gd,1,PConc,2,608.0,2.0,1786,GasA,Ex,2Story,Lvl,Gtl,Inside,IR1,162.0,CollgCr,7,Y,CompShg,Gable,Pave,6,920.0,7,6,223500
4,961,1Fam,Y,Norm,Norm,TA,TA,1,BrkTil,1,642.0,3.0,1717,GasA,Gd,2Story,Lvl,Gtl,Corner,IR1,0.0,Crawfor,7,Y,CompShg,Gable,Pave,7,756.0,91,36,140000
5,1145,1Fam,Y,Norm,Norm,TA,Gd,1,PConc,2,836.0,3.0,2198,GasA,Ex,2Story,Lvl,Gtl,FR2,IR1,350.0,NoRidge,8,Y,CompShg,Gable,Pave,9,1145.0,8,8,250000


31 32
31 32
Street           2
CentralAir       2
LotConfig        5
RoofStyle        6
Foundation       6
Heating          6
Condition2       8
RoofMatl         8
Condition1       9
Neighborhood    25
dtype: int64
Columns that must be deleted because of too much non unique values
Index(['Neighborhood'], dtype='object')

Verifying that the column of the training and testing set are the same : column difference []


## Training and testing the filtered data

In [54]:
def train_and_test(df, k=0):
    
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,subsample=0.5213, silent=1)
    
    
    parameters_for_testing = {}
    gsearch1 = GridSearchCV(estimator = lr, n_jobs=6,param_grid = parameters_for_testing,
                            verbose=50,scoring='neg_mean_squared_log_error',cv=5)
    kf = KFold(n_splits=k, shuffle=True)
    rmse_values = []
    
    train = df.iloc[:1460]
    test = df.iloc[1200:]

    gsearch1.fit(train[features], train["SalePrice"])
    predictions = gsearch1.predict(test[features])
    #mse = mean_squared_error(np.log(test["SalePrice"]), np.log(predictions))

    #rmse = np.sqrt(mse)
    #rmse_values.append(rmse)
    #print(rmse)

    #Submission file
    y_pred = gsearch1.predict(df_test[features])
    d = {"Id": df_test.index, "SalePrice": y_pred}
    sample_submission = pd.DataFrame(data = d)
    sample_submission.to_csv("Submission_naive_endk2.csv", index=False)


    #print(rmse_values)
    #avg_rmse = np.mean(rmse_values)
    avg_rmse=0
    return avg_rmse


rmse = train_and_test(transform_df, k=3)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:  1.3min
[Parallel(n_jobs=6)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.1min
[Parallel(n_jobs=6)]: Done   3 out of   5 | elapsed:  1.5min remaining:  1.0min
[Parallel(n_jobs=6)]: Done   5 out of   5 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=6)]: Done   5 out of   5 | elapsed:  1.6min finished


# Conclusion

By cleaning, filtering the data and fitting a XGB model without optimization of hyperparameters, I obtained scores between 0.14 and 0.17.  